## Deliverable 2. Create a Customer Travel Destinations Map.

In [24]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [25]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,54.0,79,0,1.01,clear sky
1,1,Victoria,HK,22.2855,114.1577,77.0,84,99,2.35,overcast clouds
2,2,Vaini,TO,-21.2000,-175.2000,78.8,89,75,12.66,broken clouds
3,3,Mataura,NZ,-46.1927,168.8643,50.0,85,84,4.00,broken clouds
4,4,Klaksvik,FO,62.2266,-6.5890,35.6,93,90,9.71,overcast clouds


In [26]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [27]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [28]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                237
City                   237
Country                237
Lat                    237
Lng                    237
Max Temp               237
Humidity               237
Cloudiness             237
Wind Speed             237
Current Description    237
dtype: int64

In [29]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Victoria,HK,22.2855,114.1577,77.00,84,99,2.35,overcast clouds
2,2,Vaini,TO,-21.2000,-175.2000,78.80,89,75,12.66,broken clouds
10,10,Hilo,US,19.7297,-155.0900,84.20,54,1,13.80,clear sky
13,13,Salalah,OM,17.0151,54.0924,82.40,78,20,1.14,few clouds
20,20,Banda Aceh,ID,5.5577,95.3222,78.39,83,68,6.22,broken clouds


In [30]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Victoria,HK,77.00,overcast clouds,22.2855,114.1577,
2,Vaini,TO,78.80,broken clouds,-21.2000,-175.2000,
10,Hilo,US,84.20,clear sky,19.7297,-155.0900,
13,Salalah,OM,82.40,few clouds,17.0151,54.0924,
20,Banda Aceh,ID,78.39,broken clouds,5.5577,95.3222,
21,Nishihara,JP,75.20,broken clouds,26.1842,127.7558,
24,Puerto Ayora,EC,82.40,few clouds,-0.7393,-90.3518,
26,Butaritari,KI,83.46,broken clouds,3.0707,172.7902,
27,Kralendijk,BQ,80.60,few clouds,12.1500,-68.2667,
30,Biak,ID,78.73,overcast clouds,-0.9131,122.8766,


In [31]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
       
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [54]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!=""]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Victoria,HK,77.00,overcast clouds,22.2855,114.1577,Mini Hotel Central
2,Vaini,TO,78.80,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
10,Hilo,US,84.20,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
13,Salalah,OM,82.40,few clouds,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH
20,Banda Aceh,ID,78.39,broken clouds,5.5577,95.3222,Sulthan Hotel International
...,...,...,...,...,...,...,...
696,Batticaloa,LK,79.23,broken clouds,7.7102,81.6924,Grand Star Guest House
699,Hirara,JP,75.20,broken clouds,24.8000,125.2833,Hotel Atoll Emerald Miyakojima
701,Lumberton,US,78.80,clear sky,34.6182,-79.0086,"Country Inn & Suites by Radisson, Lumberton, NC"
704,Tabuk,PH,77.13,scattered clouds,17.4189,121.4443,Golden Berries Hotel and Convention


In [55]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Victoria,HK,77.00,overcast clouds,22.2855,114.1577,Mini Hotel Central
2,Vaini,TO,78.80,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
10,Hilo,US,84.20,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
13,Salalah,OM,82.40,few clouds,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH
20,Banda Aceh,ID,78.39,broken clouds,5.5577,95.3222,Sulthan Hotel International
21,Nishihara,JP,75.20,broken clouds,26.1842,127.7558,Minshuku Agaihama
24,Puerto Ayora,EC,82.40,few clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
26,Butaritari,KI,83.46,broken clouds,3.0707,172.7902,Isles Sunset Lodge
27,Kralendijk,BQ,80.60,few clouds,12.1500,-68.2667,Divi Flamingo Beach Resort & Casino
30,Biak,ID,78.73,overcast clouds,-0.9131,122.8766,Penginapan Kim


In [56]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [57]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [58]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))